Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from dataclasses import dataclass
from typing import Literal, TypedDict, Callable
import math
import random
from tqdm.autonotebook import tqdm, trange


C:\Users\alex_\AppData\Local\Temp\ipykernel_6644\203297085.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## The *Nim* and *Nimply* classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        """
        Args:
            num_rows (int): number of piles
            k (int, optional): maximum number of objects nimmable each time. Defaults to None (any amount).
        """
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">" + (f" ({self._k}) " if self._k is not None else "")

    def __repr__(self):
        return self.__str__()

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    
    @property
    def k(self) -> int:
        return self._k
    

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k, f"{num_objects=}, {self._k=}"
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row]) if state._k is None else min(random.randint(1, state.rows[row]), state._k)
    return Nimply(row, num_objects)


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the smallest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c+1 if state._k is None else min(c + 1, state._k))]
    return max(possible_moves, key=lambda m: (-m[0], m[1]))

In [6]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c+1 if raw._k is None else min(c + 1, raw._k))):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    logging.debug(pformat(f"{analysis['possible_moves']}"))
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [7]:
def match(player_position: int, player_strategy, opponent, *, size: int = 5, k: int = None, lvl = logging.WARN):
    logging.getLogger().setLevel(lvl)
    strategy = (player_strategy, opponent) if player_position == 0 else (opponent, player_strategy)

    nim = Nim(size, k)
    logging.info(f"init : {nim} {bool(nim)=}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        nim.nimming(ply)
        # logging.info(f"ply: player {player} ({strategy[player].__qualname__}) \t plays {ply} -> {nim} ({nim_sum(nim)})")
        player = 1 - player
    # logging.info(f"status: Player {player} ({strategy[player].__qualname__}) won!")
    return player == player_position

In [8]:
def expert_strategy(state: Nim, klimit: bool = False) -> Nimply:
    """
    This function implement an expert systems which beats the strategies defined above
    """
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    not_zero_rows = len(state.rows) - state.rows.count(0)
    one_count_rows = state.rows.count(1)
    # if state._k is not None and klimit:
    #     non_modulo_rows = [Nimply(row, (objects % state.k + 1)) for row, objects in enumerate(state.rows) if objects > state._k and (objects % (state._k+1)) == 0]
    #     if len(non_modulo_rows) > 0:
    #         return non_modulo_rows[0]
    if one_count_rows == not_zero_rows - 1:
        is_odd = (one_count_rows % 2) == 1
        row, objects = [(row, objects) for row, objects in enumerate(state.rows) if objects > 1][0]
        if is_odd:
            return Nimply(row, objects if state.k is None else min(objects, state.k))
        return Nimply(row, objects - 1 if state.k is None else min(objects - 1, state.k))
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    logging.debug(pformat(f"{analysis['possible_moves']}"))
    ply = random.choice(spicy_moves)
    return ply

# def super_expert(state: Nim):
#     return expert_strategy(state, True)


In [9]:
# for opponent in [gabriele, pure_random, optimal]:
#     matches = 100
#     accuracy = [match(random.choice([0, 1]), expert_strategy, opponent, k=3, size=5) for _ in range(matches)].count(True)/matches
#     print(f"Accuracy of {accuracy:.2%} vs {opponent.__qualname__}")


# Adaptive Strategy

In [10]:
def remaining_moves(n: Nim, ratio: bool = False):
    mr = sum([1 for _, c in enumerate(n.rows) for _ in range(1, c+1 if n._k is None else min(c + 1, n._k))])
    if ratio:
        mt = remaining_moves(Nim(len(n.rows), n.k), False)
        return mr/mt
    else:
        return mr

In [11]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    # https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
    e_x = np.exp(x - np.max(x))
    return (e_x / e_x.sum(axis=0)).tolist() # only difference

starting_mutation_rate = (0.01, 2.5)
mutation_rate: tuple[float, float] = deepcopy(starting_mutation_rate)

@dataclass(init=False)
class Individual:
    n_strategy: int
    phase_thresholds: tuple[float] 
    strategy_probs: tuple[tuple[float]]
    _history: list
    
    def __init__(self, n_strategy: int = None, strategy_probs = None, phase_thresholds = None) -> None:
        if n_strategy is None:
            n_strategy = 4
        if strategy_probs is None:
            strategy_probs = Individual._generate_random_strategy_probs(n_strategy)
        else:
            strategy_probs = strategy_probs # if len(strategy_probs) == 3 and all([sum(strategy_probs[i]) == 1 for i in range(3)]) else Individual._softmax_probs(strategy_probs)
        if phase_thresholds is None:
            phase_thresholds = sorted([0.33, 0.66])
        else:
            phase_thresholds = sorted([max(0, phase_thresholds[0]), min(1, phase_thresholds[1])])
        
        self.n_strategy = n_strategy
        self.strategy_probs = strategy_probs
        self.phase_thresholds = phase_thresholds
        self._history: list[dict[str, int]] = [dict(),dict(),dict()] 
    
    def _generate_random_strategy_probs(n_strategy):
        matrix = [[], [], []]
        for i in range(3):
            x = [random.randint(0,10) for _ in range(n_strategy)]
            # x = softmax(x)
            matrix[i] = x
        return matrix
    def _softmax_probs(strategy_probs):
        for i in range(len(strategy_probs)):
            strategy_probs[i] = softmax(strategy_probs[i])
        return strategy_probs
    
    def mutate(ind: "Individual") -> "Individual":
        global mutation_rate
        ind = deepcopy(ind)
        version = 1
        if version == 1:
            phase_thresholds = np.random.normal(ind.phase_thresholds, mutation_rate[0]).tolist()
            strategy_probs = np.random.normal(ind.strategy_probs, mutation_rate[1]).tolist()
        else:
            phase_thresholds = deepcopy(ind.phase_thresholds)
            phase_thresholds[random.randrange(0, 2)] += np.random.normal(0, mutation_rate[0]) 
            strategy_probs = deepcopy(ind.strategy_probs)
            for i in range(ind.n_strategy):
                offset = np.random.normal(0, mutation_rate[1])
                offset_ind = random.randrange(0, ind.n_strategy)
                strategy_probs[i][offset_ind] += offset
                for other in [it for it in range(3) if it != offset_ind]:
                    strategy_probs[i][other] -= offset/2
        return Individual(strategy_probs=strategy_probs, phase_thresholds=phase_thresholds, n_strategy=ind.n_strategy)
    
    def __call__(self: "Individual", state: Nim) -> Nimply:
        phase_ratio = remaining_moves(state, True)
        phase_index = 0 if phase_ratio < self.phase_thresholds[0] else (1 if self.phase_thresholds[0] <= phase_ratio <= self.phase_thresholds[1] else 2)
        probs = softmax(self.strategy_probs[phase_index])
        STRATEGIES = [expert_strategy, gabriele, optimal, pure_random]
        strategy = np.random.choice(STRATEGIES[:self.n_strategy], p=probs)
        move = strategy(state)
        h: dict[str, int] = self._history[phase_index]
        self._history[phase_index] = {
            **h,
            strategy.__qualname__: h.get(strategy.__qualname__, 0) + 1
        }
        return move

    def reset_history(self):
        self._history = [dict(), dict(), dict()]

    @property
    def history(self: "Individual"):
        # History as percentage for each phase
        sums = [sum(phase.values()) for phase in self._history]
        ret = [dict(), dict(), dict()]
        for i in range(len(self._history)):
            for k,v in self._history[i].items():
                ret[i][k] = f"{v/sums[i]:7.2%}"
        return ret

In [12]:
ITERS = 1000
LAMBDA = 30
N_MATCHES = 10
OPPONENT = expert_strategy

In [13]:
def streak(player_strategy, n: int = None, opponent: Callable[[Nim], Nimply] = None) -> float:
    if n is None:
        n = N_MATCHES
    if opponent is None:
        opponent = OPPONENT
    wins = 0
    total = random.randrange((n*3)//4, n)
    for _ in range(total):
        random_size = random.randint(4,10)
        random_k = random.choice([None, None, *[random.randint(2, random_size*2+1) for _ in range(2)]])
        # pprint((random_size, random_k))
        wins += 1 if match(random.choice([0,1]), player_strategy, opponent, size=random_size, k=random_k) else 0
    return wins / total 

In [14]:
def train(*, variant: Literal["comma", "plus"] = "comma",
           mu: int = 1, lambda_: int = None, iters: int = None, training_factor: float = 1.1) -> TypedDict:
    if lambda_ is None:
        lambda_ = LAMBDA
    if iters is None:
        iters = ITERS
    mutation_rate = deepcopy(starting_mutation_rate)

    parents = [Individual() for _ in range(mu)]
    starting = deepcopy(parents)
    parents_result = [streak(p) for p in parents]
    pbar = trange(0, iters // lambda_, unit="epoch")
    # history_result = [parents_result]
    for _ in pbar:
        pbar.set_description(f"Training - Accuracy: {max(parents_result):.2%}")
        offspring = [(random.choice(parents)).mutate() for _ in range(lambda_)]
        
        results = [streak(i) for i in tqdm(offspring, unit="streak", leave=False, disable=True)]
        
        incrate = (np.sum([res > sum(parents_result)/len(parents_result) for res in results])/lambda_)

        if incrate > 1/5:
            mutation_rate = (mutation_rate[0]*training_factor, mutation_rate[1]*training_factor)
        elif incrate < 1/5:
            mutation_rate = (mutation_rate[0]/training_factor, mutation_rate[1]/training_factor)

        
        population = list(zip(results, offspring))
        if variant == "plus":
            population.extend(list(zip(parents_result, parents)))
        population = sorted(population, key=lambda i:i[0], reverse=True)[:mu]

        parents = [it[1] for it in population]
        parents_result = [it[0] for it in population]
        
    best_ind = np.argmax(parents_result)

    return {
        "best": (parents_result[best_ind], parents[best_ind]),
    	"starting": starting,
        "parents": zip(parents_result, parents)
    }

def evaluate(ind: Individual, name: str = None):
    ind.reset_history()
    opponents = [gabriele, pure_random, optimal, expert_strategy]
    if name is None:
        name = ""
    acc_onecomma = list(zip(
        [streak(ind, 100, opponent) for opponent in tqdm(opponents, leave=False, desc=f"Evaluating {name}", unit="opponent")],
        [it.__qualname__ for it in opponents])
        )
    msg = "Accuracy of" if name == "" else f"{name} has an accuracy of"
    print("\n".join([f"{msg} {acc:6.2%} vs {o}" for acc,o in acc_onecomma]))
    print(f"History: {pformat(ind.history)}")
    print(f"StrategyProbs: {pformat([[f'{itit:.3f}' for itit in it ] for it in ind.strategy_probs])}")
    print(f"Thresholds: {pformat([f'{it:.3f}' for it in ind.phase_thresholds])}")


# $(1,\lambda)$ - ES

In [15]:
res = train(variant="comma", mu=1)
ind_onecomma: Individual
_, ind_onecomma = res["best"]

  0%|          | 0/33 [00:00<?, ?epoch/s]

In [16]:
evaluate(ind_onecomma, "One-Comma")

Evaluating One-Comma:   0%|          | 0/4 [00:00<?, ?opponent/s]

One-Comma has an accuracy of 97.98% vs gabriele
One-Comma has an accuracy of 95.40% vs pure_random
One-Comma has an accuracy of 97.70% vs optimal
One-Comma has an accuracy of 43.68% vs expert_strategy
History: [{'expert_strategy': ' 99.76%', 'pure_random': '  0.24%'},
 {'expert_strategy': ' 99.86%', 'optimal': '  0.14%'},
 {'expert_strategy': '100.00%'}]
StrategyProbs: [['33.784', '-11.496', '5.726', '26.985'],
 ['15.761', '4.808', '5.523', '-4.150'],
 ['9.565', '-2.409', '-9.258', '-10.164']]
Thresholds: ['0.361', '0.598']


## $(\mu, \lambda)$ - ES

In [17]:
res = train(variant="comma", mu=10)
ind_mucomma: Individual
_, ind_mucomma = res["best"]

  0%|          | 0/33 [00:00<?, ?epoch/s]

In [18]:
evaluate(ind_mucomma, "μ-Comma")


Evaluating μ-Comma:   0%|          | 0/4 [00:00<?, ?opponent/s]

μ-Comma has an accuracy of 96.74% vs gabriele
μ-Comma has an accuracy of 94.85% vs pure_random
μ-Comma has an accuracy of 98.95% vs optimal
μ-Comma has an accuracy of 12.94% vs expert_strategy
History: [{'expert_strategy': '100.00%'},
 {'pure_random': '100.00%'},
 {'expert_strategy': ' 98.35%', 'pure_random': '  1.65%'}]
StrategyProbs: [['41.456', '13.682', '-5.240', '26.948'],
 ['-0.224', '-13.167', '16.459', '22.260'],
 ['19.520', '9.990', '7.453', '15.183']]
Thresholds: ['0.419', '0.669']


# $(1+\lambda)$ - ES

In [19]:
res = train(variant="plus", mu=1)
ind_oneplus: Individual
_, ind_oneplus = res["best"]

  0%|          | 0/33 [00:00<?, ?epoch/s]

In [20]:
evaluate(ind_oneplus, "One-Plus")

Evaluating One-Plus:   0%|          | 0/4 [00:00<?, ?opponent/s]

One-Plus has an accuracy of 72.50% vs gabriele
One-Plus has an accuracy of 42.67% vs pure_random
One-Plus has an accuracy of 28.28% vs optimal
One-Plus has an accuracy of  2.50% vs expert_strategy
History: [{'expert_strategy': '  0.10%', 'optimal': '  0.50%', 'pure_random': ' 99.40%'},
 {'gabriele': ' 96.73%', 'optimal': '  3.02%', 'pure_random': '  0.25%'},
 {'expert_strategy': ' 94.81%', 'pure_random': '  5.19%'}]
StrategyProbs: [['6.858', '1.898', '6.913', '12.478'],
 ['-2.001', '11.728', '8.023', '5.754'],
 ['11.983', '1.868', '2.577', '9.141']]
Thresholds: ['0.347', '0.640']


# $(\mu+\lambda)$ - ES

In [21]:
res = train(variant="plus", mu=10)
ind_muplus: Individual
_, ind_muplus = res["best"]

  0%|          | 0/33 [00:00<?, ?epoch/s]

In [22]:
evaluate(ind_muplus, "μ-Plus")

Evaluating μ-Plus:   0%|          | 0/4 [00:00<?, ?opponent/s]

μ-Plus has an accuracy of 97.92% vs gabriele
μ-Plus has an accuracy of 95.70% vs pure_random
μ-Plus has an accuracy of 98.96% vs optimal
μ-Plus has an accuracy of 60.76% vs expert_strategy
History: [{'expert_strategy': ' 99.89%', 'gabriele': '  0.11%'},
 {'expert_strategy': ' 99.92%', 'gabriele': '  0.08%'},
 {'expert_strategy': ' 99.70%', 'optimal': '  0.30%'}]
StrategyProbs: [['9.514', '3.921', '-0.438', '-14.999'],
 ['13.423', '4.876', '-14.709', '5.955'],
 ['15.732', '-1.106', '10.755', '-0.125']]
Thresholds: ['0.283', '0.722']


# Extensions

## Genetic Approach

In [23]:
# This is the start of the code when i thought i was supposed to do a Genetic Algorithm

from dataclasses import dataclass, asdict, field
from typing import Literal, TypedDict, Any
import random

Allele: TypedDict = {
    "prefer_rows": {"type": "discrete", "value": [0, 1, -1]},
    "percent_to_take": {"type": "continous", "value": [0, 1]},
}

def random_allele_value(key):
    v: dict[str, Any] = Allele[key]
    if v.get("type") == "discrete":
        return random.choice(v["value"])
    elif v.get("type") == "continous":
        start, stop = v.get("value")
        size = stop-start
        return (random.random() * size) + start


@dataclass(frozen=True)
class Genome:
    prefer_rows: Literal[0, 1, -1] = field(default_factory=lambda: random_allele_value("prefer_rows"))
    """-1 favours smaller rows, 1 bigger, 0 indifferent"""
    percent_to_take: float = field(default_factory=lambda: random_allele_value("percent_to_take"))
    """Range: [0, 1]"""

    def mutate(g1: "Genome") -> "Genome":
        """Alters one single gene of the starting genome

        Args:
            g1 (Genome): Starting genome (never altered)

        Returns:
            Genome: Mutated genome
        """
        d = asdict(g1)
        rand_attr_to_change: str = random.choice(list(d.keys()))
        d[rand_attr_to_change] = random_allele_value(rand_attr_to_change)
        return Genome(**d)

    def crossover(g1: "Genome", g2: "Genome") -> "Genome":
        d1, d2 = asdict(g1), asdict(g2)
        child = dict()
        for field in d1.keys():
            child[field] = d1[field] if random.random() < 0.5 else d2[field]

        return Genome(**child)
    
d = Genome()
e = Genome()
print(d, e, d.crossover(e).mutate())

Genome(prefer_rows=1, percent_to_take=0.5934864392811888) Genome(prefer_rows=0, percent_to_take=0.5931051126001485) Genome(prefer_rows=-1, percent_to_take=0.5931051126001485)
